In [2]:
from bibleabbr import *

In [3]:
import numpy as np
import shorttext
import sqlite3

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Scriptures Loaded

In [9]:
dbconn = sqlite3.connect('ESVCrossway.sqlite')

In [10]:
from functools import reduce
from nltk import sent_tokenize

def retrieve_docs_as_biblechapters(dbconn):
    cursor = dbconn.cursor()
    for book in wholebible_book_iterator():
        for chap in range(1, numchaps[book]+1):
            doc_label = book+'_'+str(chap)
            result = cursor.execute('select scripture from bible where book is "'+book+'" and chapter='+str(chap))
            doc_text = reduce(lambda s1, s2: ' '.join([s1, s2]), [texttuple[0] for texttuple in result])
            yield doc_label, doc_text
    cursor.close()
    

In [11]:
from collections import defaultdict

classdict = defaultdict(lambda : [])
for bible_chap, verses in retrieve_docs_as_biblechapters(dbconn):
    for verse in sent_tokenize(verses):
        classdict[bible_chap] += [verse]

# Topic Models

In [20]:
from shorttext.generators import LDAModeler

nbs_topics = [10, 100]

topicmodelers = {nbtopics: LDAModeler() for nbtopics in nbs_topics}
for nbtopics in topicmodelers:
    print('Training nbtopics={}'.format(nbtopics))
    topicmodeler = topicmodelers[nbtopics]
    topicmodeler.train(classdict, nbtopics)

Training nbtopics=10
Training nbtopics=100


In [21]:
topicmodelers[10].save_compact_model('biblechap_lda10.pkl')
topicmodelers[100].save_compact_model('biblechap_lda100.pkl')

/usr/local/lib/python3.5/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.5/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.5/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'Se

# Gaussian Naive Bayes

In [5]:
topicmodel_tp10 = shorttext.smartload_compact_model('biblechap_lda10.pkl', None)

/usr/local/lib/python3.5/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.5/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [6]:
from sklearn.naive_bayes import GaussianNB

In [7]:
gaunb_classifier_tp10 = shorttext.classifiers.TopicVectorSkLearnClassifier(topicmodel_tp10, GaussianNB())

In [13]:
gaunb_classifier_tp10.train(classdict)